Python program to open a fasta file, parse it and then return as sequence

In [55]:
def read_fasta(file_path):
    sequences = {}
    try:
        with open(file_path, 'r') as file:
            identifier = None
            sequence = []
            for line in file:
                line = line.strip()
                if line.startswith('>'):
                    if identifier:
                        sequences[identifier] = ''.join(sequence)
                    identifier = line[1:].split()[0]
                    sequence = []
                else:
                    sequence.append(line)
            if identifier:
                sequences[identifier] = ''.join(sequence)
    except Exception as e:
        print(f"Error reading file: {e}")
    return sequences

(2) What are the lengths of the sequences in the file? What is the longest sequence and what is the shortest sequence? Is there more than one longest or shortest sequence? What are their identifiers?

In [35]:
def count_records(sequences):
    return len(sequences)

In [49]:
def sequence_lengths(sequences):
    lengths = {identifier: len(seq) for identifier, seq in sequences.items()}
    max_length = max(lengths.values())
    min_length = min(lengths.values())
    longest_sequences = [identifier for identifier, length in lengths.items() if length == max_length]
    shortest_sequences = [identifier for identifier, length in lengths.items() if length == min_length]
    return lengths, longest_sequences, shortest_sequences

(3) In molecular biology, a reading frame is a way of dividing the DNA sequence of nucleotides into a set of consecutive, non-overlapping triplets (or codons). Depending on where we start, there are six possible reading frames: three in the forward (5' to 3') direction and three in the reverse (3' to 5'). For instance, the three possible forward reading frames for the sequence AGGTGACACCGCAAGCCTTATATTAGC are: 

AGG TGA CAC CGC AAG CCT TAT ATT AGC

A GGT GAC ACC GCA AGC CTT ATA TTA GC

AG GTG ACA CCG CAA GCC TTA TAT TAG C 

These are called reading frames 1, 2, and 3 respectively. An open reading frame (ORF) is the part of a reading frame that has the potential to encode a protein. It starts with a start codon (ATG), and ends with a stop codon (TAA, TAG or TGA). For instance, ATGAAATAG is an ORF of length 9.

Given an input reading frame on the forward strand (1, 2, or 3) your program should be able to identify all ORFs present in each sequence of the FASTA file, and answer the following questions: what is the length of the longest ORF in the file? What is the identifier of the sequence containing the longest ORF? For a given sequence identifier, what is the longest ORF contained in the sequence represented by that identifier? What is the starting position of the longest ORF in the sequence that contains it? The position should indicate the character number in the sequence. For instance, the following ORF in reading frame 1:

'>sequence1'

ATGCCCTAG

starts at position 1.

Note that because the following sequence:

'>sequence2'

ATGAAAAAA

does not have any stop codon in reading frame 1, we do not consider it to be an ORF in reading frame 1.

In [62]:
def find_orfs(sequence, frame=1):
    """ Finds all open reading frames (ORFs) in a given sequence and frame.
        Returns a list of tuples (start_position, end_position, length). """
    start_codon = 'ATG'
    stop_codons = ['TAA', 'TAG', 'TGA']
    seq_len = len(sequence)
    orfs = []
    for i in range(frame - 1, seq_len, 3):  # Adjust frame to 0-based index
        codon = sequence[i:i+3]
        if codon == start_codon:
            for j in range(i+3, seq_len, 3):
                codon = sequence[j:j+3]
                if codon in stop_codons:
                    orfs.append((i+1, j+3, j+3 - i))  # +1 to convert 0-based to 1-based index
                    break
    return orfs

def find_longest_orf_in_frame(sequences, frame):
    longest_orf_length = 0
    longest_orf_id = None
    longest_orf_start = None
    
    for identifier, sequence in sequences.items():
        orfs = find_orfs(sequence, frame)
        for start, end, length in orfs:
            if length > longest_orf_length:
                longest_orf_length = length
                longest_orf_id = identifier
                longest_orf_start = start
    
    return longest_orf_length, longest_orf_id, longest_orf_start

4) A repeat is a substring of a DNA sequence that occurs in multiple copies (more than one) somewhere in the sequence. Although repeats can occur on both the forward and reverse strands of the DNA sequence, we will only consider repeats on the forward strand here. Also we will allow repeats to overlap themselves. For example, the sequence ACACA contains two copies of the sequence ACA - once at position 1 (index 0 in Python), and once at position 3. Given a length n, your program should be able to identify all repeats of length n in all sequences in the FASTA file. Your program should also determine how many times each repeat occurs in the file, and which is the most frequent repeat of a given length.


In [60]:
def find_repeats(sequence, n):
    """ Finds all repeats of length n in a given sequence.
        Returns a dictionary where keys are repeats and values are their counts. """
    repeats = {}
    seq_len = len(sequence)
    
    for i in range(seq_len - n + 1):
        repeat = sequence[i:i+n]
        if repeat in repeats:
            repeats[repeat] += 1
        else:
            repeats[repeat] = 1
    
    return repeats

def find_most_frequent_repeat(sequences, n):
    """ Finds the most frequent repeat of length n across all sequences.
        Returns a tuple (repeat, count). """
    all_repeats = {}
    
    for identifier, sequence in sequences.items():
        repeats = find_repeats(sequence, n)
        for repeat, count in repeats.items():
            if repeat in all_repeats:
                all_repeats[repeat] += count
            else:
                all_repeats[repeat] = count
    
    if all_repeats:
        most_frequent_repeat = max(all_repeats, key=all_repeats.get)
        return most_frequent_repeat, all_repeats[most_frequent_repeat]
    else:
        return None, 0

Now to process the main function using the program

In [65]:
def main(file_path, frame, repeat_length):
    sequences = read_fasta(file_path)
    
    
    if not sequences:
        print("No sequences found. Please check the input file.")
        return
    
    # Question 1
    num_records = count_records(sequences)
    print(f"Number of records: {num_records}")
    
    # Question 2
    lengths, longest_sequences, shortest_sequences = sequence_lengths(sequences)
    print(f"Lengths of sequences: {lengths}")
    print(f"Longest sequences: {longest_sequences}")
    print(f"Shortest sequences: {shortest_sequences}")
    
    # Question 3
    longest_orf_length, longest_orf_id, longest_orf_start = find_longest_orf(sequences, frame)
    print(f"Longest ORF length: {longest_orf_length}")
    print(f"Identifier of sequence with longest ORF: {longest_orf_id}")
    print(f"Starting position of longest ORF: {longest_orf_start}")
    
    # Question 4
    most_frequent, count = find_most_frequent_repeat(sequences, repeat_length)
    print(f"Most frequent repeat of length {repeat_length}: {most_frequent} (occurs {count} times)")

if __name__ == "__main__":
    file_path = "dna2.fasta"  # Replace with the actual file path for the exam
    frame = 2  # Replace with the actual reading frame for the exam
    repeat_length = 3  # Replace with the actual repeat length for the exam
    main(file_path, frame, repeat_length)


Number of records: 18
Lengths of sequences: {'gi|142022655|gb|EQ086233.1|91': 4635, 'gi|142022655|gb|EQ086233.1|304': 1151, 'gi|142022655|gb|EQ086233.1|255': 4894, 'gi|142022655|gb|EQ086233.1|45': 3511, 'gi|142022655|gb|EQ086233.1|396': 4076, 'gi|142022655|gb|EQ086233.1|250': 2867, 'gi|142022655|gb|EQ086233.1|322': 442, 'gi|142022655|gb|EQ086233.1|88': 890, 'gi|142022655|gb|EQ086233.1|594': 967, 'gi|142022655|gb|EQ086233.1|293': 4338, 'gi|142022655|gb|EQ086233.1|75': 1352, 'gi|142022655|gb|EQ086233.1|454': 4564, 'gi|142022655|gb|EQ086233.1|16': 4804, 'gi|142022655|gb|EQ086233.1|584': 964, 'gi|142022655|gb|EQ086233.1|4': 2095, 'gi|142022655|gb|EQ086233.1|277': 1432, 'gi|142022655|gb|EQ086233.1|346': 115, 'gi|142022655|gb|EQ086233.1|527': 2646}
Longest sequences: ['gi|142022655|gb|EQ086233.1|255']
Shortest sequences: ['gi|142022655|gb|EQ086233.1|346']
Longest ORF length: 1458
Identifier of sequence with longest ORF: gi|142022655|gb|EQ086233.1|16
Starting position of longest ORF: 3071
Mos

In [66]:
def read_fasta(file_path):
    sequences = {}
    try:
        with open(file_path, 'r') as file:
            identifier = None
            sequence = []
            for line in file:
                line = line.strip()
                if line.startswith('>'):
                    if identifier:
                        sequences[identifier] = ''.join(sequence)
                    identifier = line[1:].split()[0]
                    sequence = []
                else:
                    sequence.append(line)
            if identifier:
                sequences[identifier] = ''.join(sequence)
    except Exception as e:
        print(f"Error reading file: {e}")
    return sequences

def find_orfs(sequence, frame=1):
    """ Finds all open reading frames (ORFs) in a given sequence and frame.
        Returns a list of tuples (start_position, end_position, length). """
    start_codon = 'ATG'
    stop_codons = ['TAA', 'TAG', 'TGA']
    seq_len = len(sequence)
    orfs = []
    for i in range(frame - 1, seq_len, 3):  # Adjust frame to 0-based index
        codon = sequence[i:i+3]
        if codon == start_codon:
            for j in range(i+3, seq_len, 3):
                codon = sequence[j:j+3]
                if codon in stop_codons:
                    orfs.append((i+1, j+3, j+3 - i))  # +1 to convert 0-based to 1-based index
                    break
    return orfs

def find_longest_orf(sequence):
    longest_orf_length = 0
    longest_orf_start = None
    longest_orf_end = None
    
    for frame in range(1, 4):
        orfs = find_orfs(sequence, frame)
        for start, end, length in orfs:
            if length > longest_orf_length:
                longest_orf_length = length
                longest_orf_start = start
                longest_orf_end = end
    
    return longest_orf_length, longest_orf_start, longest_orf_end

# Example usage for the specific sequence
file_path = "dna2.fasta"  # Replace with the actual file path for the exam
sequences = read_fasta(file_path)
identifier = 'gi|142022655|gb|EQ086233.1|16'
sequence = sequences.get(identifier)

if sequence:
    longest_orf_length, longest_orf_start, longest_orf_end = find_longest_orf(sequence)
    print(f"Longest ORF length in sequence {identifier}: {longest_orf_length}")
    print(f"Starting position of longest ORF in sequence {identifier}: {longest_orf_start}")
    print(f"Ending position of longest ORF in sequence {identifier}: {longest_orf_end}")
else:
    print(f"Sequence with identifier {identifier} not found.")

Longest ORF length in sequence gi|142022655|gb|EQ086233.1|16: 1644
Starting position of longest ORF in sequence gi|142022655|gb|EQ086233.1|16: 1440
Ending position of longest ORF in sequence gi|142022655|gb|EQ086233.1|16: 3083


In [67]:
from collections import defaultdict

def read_fasta(file_path):
    sequences = {}
    try:
        with open(file_path, 'r') as file:
            identifier = None
            sequence = []
            for line in file:
                line = line.strip()
                if line.startswith('>'):
                    if identifier:
                        sequences[identifier] = ''.join(sequence)
                    identifier = line[1:].split()[0]
                    sequence = []
                else:
                    sequence.append(line)
            if identifier:
                sequences[identifier] = ''.join(sequence)
    except Exception as e:
        print(f"Error reading file: {e}")
    return sequences

def find_most_frequent_repeat(sequences, repeat_length):
    repeat_counts = defaultdict(int)
    
    for sequence in sequences.values():
        seq_len = len(sequence)
        for i in range(seq_len - repeat_length + 1):
            repeat = sequence[i:i + repeat_length]
            repeat_counts[repeat] += 1
    
    most_frequent_repeat = max(repeat_counts, key=repeat_counts.get)
    most_frequent_count = repeat_counts[most_frequent_repeat]
    
    return most_frequent_repeat, most_frequent_count

# Example usage for repeat length 6
file_path = "dna2.fasta"  # Replace with the actual file path for the exam
sequences = read_fasta(file_path)
repeat_length = 6
most_frequent_repeat, most_frequent_count = find_most_frequent_repeat(sequences, repeat_length)
print(f"Most frequently occurring repeat of length {repeat_length}: {most_frequent_repeat}")
print(f"Number of times it occurs: {most_frequent_count}")

Most frequently occurring repeat of length 6: GCGCGC
Number of times it occurs: 153


In [68]:
from collections import defaultdict

def read_fasta(file_path):
    sequences = {}
    try:
        with open(file_path, 'r') as file:
            identifier = None
            sequence = []
            for line in file:
                line = line.strip()
                if line.startswith('>'):
                    if identifier:
                        sequences[identifier] = ''.join(sequence)
                    identifier = line[1:].split()[0]
                    sequence = []
                else:
                    sequence.append(line)
            if identifier:
                sequences[identifier] = ''.join(sequence)
    except Exception as e:
        print(f"Error reading file: {e}")
    return sequences

def find_repeats(sequences, repeat_length):
    repeat_counts = defaultdict(int)
    
    for sequence in sequences.values():
        seq_len = len(sequence)
        for i in range(seq_len - repeat_length + 1):
            repeat = sequence[i:i + repeat_length]
            repeat_counts[repeat] += 1
    
    return repeat_counts

def find_max_repeats(repeat_counts):
    max_count = max(repeat_counts.values())
    max_repeats = [repeat for repeat, count in repeat_counts.items() if count == max_count]
    return max_count, max_repeats

# Example usage for repeat length 12
file_path = "dna2.fasta"  # Replace with the actual file path for the exam
sequences = read_fasta(file_path)
repeat_length = 12
repeat_counts = find_repeats(sequences, repeat_length)
max_count, max_repeats = find_max_repeats(repeat_counts)

print(f"Maximum number of copies of the most frequent repeat of length {repeat_length}: {max_count}")
print(f"Number of different 12-base sequences that occur {max_count} times: {len(max_repeats)}")

Maximum number of copies of the most frequent repeat of length 12: 10
Number of different 12-base sequences that occur 10 times: 4


In [69]:
from collections import defaultdict

def read_fasta(file_path):
    sequences = {}
    try:
        with open(file_path, 'r') as file:
            identifier = None
            sequence = []
            for line in file:
                line = line.strip()
                if line.startswith('>'):
                    if identifier:
                        sequences[identifier] = ''.join(sequence)
                    identifier = line[1:].split()[0]
                    sequence = []
                else:
                    sequence.append(line)
            if identifier:
                sequences[identifier] = ''.join(sequence)
    except Exception as e:
        print(f"Error reading file: {e}")
    return sequences

def count_specific_repeats(sequences, repeat_length, repeats_to_check):
    repeat_counts = defaultdict(int)
    
    for sequence in sequences.values():
        seq_len = len(sequence)
        for i in range(seq_len - repeat_length + 1):
            repeat = sequence[i:i + repeat_length]
            if repeat in repeats_to_check:
                repeat_counts[repeat] += 1
    
    return repeat_counts

# Example usage for repeat length 7
file_path = "dna2.fasta"  # Replace with the actual file path for the exam
sequences = read_fasta(file_path)
repeat_length = 7
repeats_to_check = ["TGCGCGC", "CATCGCC", "CGCGCCG", "GCGCGCA"]
repeat_counts = count_specific_repeats(sequences, repeat_length, repeats_to_check)

# Find the repeat with the maximum number of occurrences
max_repeat = max(repeat_counts, key=repeat_counts.get)
max_count = repeat_counts[max_repeat]

print(f"Repeat of length {repeat_length} with the maximum number of occurrences: {max_repeat}")
print(f"Number of occurrences: {max_count}")

Repeat of length 7 with the maximum number of occurrences: CGCGCCG
Number of occurrences: 63
